In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

from bert_score import BERTScorer


In [1]:

from bert_score import BERTScorer

ValueError: libcublas.so.*[0-9] not found in the system path ['/mnt/d/Projects/papersurvey_tool/notebooks', '/python', '/mnt/d/Projects/papersurvey_tool/notebooks', '/home/alexis/.pyenv/versions/3.10.12/lib/python310.zip', '/home/alexis/.pyenv/versions/3.10.12/lib/python3.10', '/home/alexis/.pyenv/versions/3.10.12/lib/python3.10/lib-dynload', '', '/mnt/d/Projects/papersurvey_tool/.venv/lib/python3.10/site-packages']